In [3]:
from sympy import *

In [4]:
#variable definitions
t1, t2, l, q, m1, m2, b1, b2  = symbols('t_1 t_2 l q m_1 m_2 b_1 b_2')

init_printing(use_unicode=True)

In [5]:
#profits
pi1 = .5*( (t2-t1)/3.0 +q)*(1+ (1/l)*(t2-t1)/3.0 )
pi2 = .5*( (t1-t2)/3.0 +q)*(1+ (1/l)*(t1-t2)/3.0 )

#nash bargaining objectives
v1 = (pi1 - pi1.subs(t1,l) ) * (m1*t1 +m2*t2)
v2 = (pi2 - pi2.subs(t2,l) ) * (m1*t1 +m2*t2)

#start with out b1 and b2 for ease of computation

In [7]:
#test code
x = Symbol('x')
solve(x**2-1,x)

In [13]:
#simultaneous bargaining
v1_foc = diff(v1,t1)
v2_foc = diff(v2,t2)

#solve for t1 and t2
print v1_foc
print '-----'
t1_foc = solve(v1_foc,t1)

#only positive solution
t1_foc = t1_foc[1]

v2_foc2 = v2_foc.subs(t1,t1_foc)
print v2_foc2 
print '-----'

#t2_param = solve(v2_foc2,t2)
#print t2_param
#print '-----'

m_1*(-(1 + 0.333333333333333*(-l + t_2)/l)*(-0.166666666666667*l + 0.5*q + 0.166666666666667*t_2) + (1 + 0.333333333333333*(-t_1 + t_2)/l)*(0.5*q - 0.166666666666667*t_1 + 0.166666666666667*t_2)) + (m_1*t_1 + m_2*t_2)*(-0.166666666666667 - 0.0555555555555556*(-t_1 + t_2)/l - 0.333333333333333*(0.5*q - 0.166666666666667*t_1 + 0.166666666666667*t_2)/l)
-----
m_2*(-(1 + 0.333333333333333*(q + 0.666666666666667*t_2 - 0.333333333333333*m_2*t_2/m_1 + 0.333333333333333*sqrt(3.0*l**2*m_1**2 + 9.0*l*m_1**2*q + 6.0*l*m_1**2*t_2 + 3.0*l*m_1*m_2*t_2 + 9.0*m_1**2*q**2 + 12.0*m_1**2*q*t_2 + 4.0*m_1**2*t_2**2 + 3.0*m_1*m_2*q*t_2 + 2.0*m_1*m_2*t_2**2 + m_2**2*t_2**2)/m_1)/l)*(0.666666666666667*q + 0.111111111111111*t_2 - 0.0555555555555556*m_2*t_2/m_1 + 0.0555555555555556*sqrt(3.0*l**2*m_1**2 + 9.0*l*m_1**2*q + 6.0*l*m_1**2*t_2 + 3.0*l*m_1*m_2*t_2 + 9.0*m_1**2*q**2 + 12.0*m_1**2*q*t_2 + 4.0*m_1**2*t_2**2 + 3.0*m_1*m_2*q*t_2 + 2.0*m_1*m_2*t_2**2 + m_2**2*t_2**2)/m_1) + (1 + 0.333333333333333*(l + q - 0

In [ ]:

v2_foc = sympy.diff(v2,t2)
t1_foc = t1_foc.subs({l:2, q:2, m1:.5, m2:.5})
t1_foc = sympy.simplify(t1_foc)
print '-----'
print t1_foc

t1_foc = t1_foc.subs({l:2, q:2, m1:.5, m2:.5})
t1_foc = sympy.simplify(t1_foc)
print '-----'
print t1_foc

In [132]:
# try to get t1 into t2
v2_foc2 = v2_foc.subs(t1,t1_foc)
#v2_foc2 = v2_foc2.subs({l:2, q:2, m1:.5, m2:.5})
#v2_foc = v2_foc.subs({l:2, q:2, m1:.5, m2:.5})
#print v2_foc2
#v2_foc2

AttributeError: 'FiniteSet' object has no attribute 'as_coeff_Mul'

AttributeError: 'FiniteSet' object has no attribute 'as_coeff_Mul'

AttributeError: 'FiniteSet' object has no attribute 'as_coeff_Mul'

In [36]:
#t1 and t2 in terms of model parameters
t2_param = sympy.solve(v2_foc2,t2)
print '------'
print t2_params
print t2_params.subs(l,2)
t1_params = sympy.simplify(t2_params.subs(t1_foc,t2_foc))
print t2_params

[3.0*l + t_2]


In [ ]:
#mmisc stuff for later
pi_foc = sympy.diff(pi_1,t_1)
print pi_foc

h_t2 = sympy.solve(pi_foc,t_1)
print h_t2